In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/VRD-IU

In [ ]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import pickle
from transformers import AutoTokenizer
import torch
class CompTextDataset(Dataset):
    def __init__(self, pickle_file):
        super().__init__()
        with open(pickle_file, 'rb') as file:
            data = pickle.load(file)
        self.components = []
        for k in data.keys():
            for comp in data[k]['components']:
                if comp['bbox'] == [0.0, 0.0, 0.0, 0.0]:
                  continue
                self.components.append(comp)

    def __len__(self):
        return len(self.components)

    def __getitem__(self, index):
        comp = self.components[index]
        try:
            text = comp['text']
        except:
            text = comp['category']
        return text, comp['object_id']

In [ ]:
train_dataset = CompVisualDataset('train_data.pkl')

In [ ]:
from transformers import XLMRobertaModel
model = XLMRobertaModel.from_pretrained("FacebookAI/xlm-roberta-base")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using: {device}")
model.to(device)
model.eval()

In [ ]:

tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")

In [ ]:
from tqdm import tqdm
import os
def extract_features(dataloader, feature_path):
  if not os.path.exists(feature_path):
    os.makedirs(feature_path)
  with torch.no_grad():
      for texts, object_ids in tqdm(dataloader):
          text_inputs = tokenizer(texts, return_tensors="pt",padding=True, truncation=True).to(device)
          outputs = model(**text_inputs)
          features = outputs.pooler_output.detach().cpu()
          for idx, obj_id in enumerate(object_ids):
            torch.save(features[idx],os.path.join(feature_path,f"{obj_id}.pt"))

In [ ]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [ ]:
train_dataloader = DataLoader(subset,batch_size=256, collate_fn= collate_fn, num_workers=6)

In [ ]:
extract_features(train_dataloader,  'train_textual_features')
print("Extraction completed for training set!")